In [ ]:
import torch_ops
import data_pipes
import constants
import yfinance as yf

# Get symbols from constants file
lines = constants.sa_str.splitlines()
symbols = [line.split("\t")[1] for line in lines][:100]

# Download data
df = yf.download(symbols + ["SPY"], period="1y", interval="1h", ignore_tz=True)

data = data_pipes.nova_data(symbols, df)

In [10]:
import torch
trim_ratio = 0.99
split_ratio = 0.999

x_base = data["x"][int(trim_ratio * len(data["x"])):]
y_base = data["y"][int(trim_ratio * len(data["y"])):]

x_t_np = x_base[:int(split_ratio * len(x_base))]
x_v_np = x_base[int(split_ratio * len(x_base)):]
y_t_np = y_base[:int(split_ratio * len(y_base))]
y_v_np = y_base[int(split_ratio * len(y_base)):]

x_t = torch.tensor(x_t_np, dtype=torch.float32)
x_v = torch.tensor(x_v_np, dtype=torch.float32)
y_t = torch.tensor(y_t_np, dtype=torch.float32)
y_v = torch.tensor(y_v_np, dtype=torch.float32)

In [ ]:
from torch import nn, optim
from copy import deepcopy

class nova_0(nn.Module):
    def __init__(self, num_symbols, reduction_factor=1):
        super().__init__()
        self.hidden0 = nn.Linear(num_symbols, num_symbols // reduction_factor)
        self.act0 = nn.SELU()
        self.hidden1 = nn.Linear(num_symbols // reduction_factor, num_symbols // reduction_factor)
        self.act1 = nn.SELU()
        self.hidden2 = nn.Linear(num_symbols // reduction_factor, num_symbols // reduction_factor)
        self.act2 = nn.SELU()
        self.hidden3 = nn.Linear(num_symbols // reduction_factor, num_symbols)
        self.out = nn.Tanh()
 
    def forward(self, x):
        x = self.act0(self.hidden0(x))
        x = self.act1(self.hidden1(x))
        # x = self.act2(self.hidden2(x))
        x = self.out(self.hidden3(x))
        # x = self.hidden3(x)
        return x

nova = nova_0(x_t.shape[1])
epochs, batch_size, lr = 1000, x_t.shape[0]//2, 5e-5
optimizer = optim.NAdam(nova.parameters(), lr=lr)
loss_fn = nn.L1Loss()

test_loss = 999
for epoch in range(epochs):
    loss = torch_ops.train_loop(x_t, y_t, nova, loss_fn, optimizer, batch_size)
    test_loss_new = torch_ops.test_loop(x_v, y_v, nova, loss_fn, batch_size)
    if test_loss_new < test_loss or epoch == epochs - 1:
        print(f"---------- Epoch {epoch + 1} ----------")
        print(f"loss: {loss:1.5f}, test loss: {test_loss_new:1.5f}")
        
    if test_loss_new < test_loss:
        test_loss = test_loss_new
        print(f"Checkpoint: test loss = {test_loss:1.5f} <---------------")
        state_dict_save = deepcopy(nova.state_dict())

print("Done.")

